model metrics f1 score, accuracy
model parameters -> dataset version, solver i.e. adamw, activation function

# 1) Quick path (works if your model’s `forward` accepts a NumPy array / tensor directly)

**A. Start MLflow tracking server (optional but recommended)**

```bash
mlflow server \
  --host 127.0.0.1 --port 5000 \
  --backend-store-uri sqlite:///mlflow.db \
  --artifacts-destination ./artifacts
```

**B. Log + register the PyTorch model**

```python
import torch, mlflow, mlflow.pytorch
import numpy as np
from mlflow.models import infer_signature

# 1) Recreate your model and load weights
model = YourModelClass(...)
state = torch.load("your_model.pth", map_location="cpu")
model.load_state_dict(state)
model.eval()

# 2) Create a sample input matching your real inference input
# Example: (N, C, H, W) = (1,1,224,224)
x_example = np.random.rand(1, 1, 224, 224).astype("float32")

# 3) Infer signature (optional but strongly recommended)
with torch.no_grad():
    y_example = model(torch.from_numpy(x_example)).numpy()
signature = infer_signature(x_example, y_example)

mlflow.set_tracking_uri("http://127.0.0.1:5000")  # if running a server
registered_name = "brain_mri_classifier"          # pick your name

with mlflow.start_run():
    mlflow.log_param("model_arch", "YourModelClass")
    mlflow.pytorch.log_model(
        pytorch_model=model,
        artifact_path="model",
        registered_model_name=registered_name,
        signature=signature,
        input_example=x_example,
        pip_requirements=[
            "mlflow==2.14.1",        # use your versions
            "torch==2.3.1",
            "numpy",
            "pandas",
            "pillow"
        ],
    )
```






**D. Serve the Production model**

```bash
mlflow models serve \
  -m "models:/brain_mri_classifier/Production" \
  -h 0.0.0.0 -p 8080
```

**E. Call the REST API**

```bash
curl -X POST http://127.0.0.1:8080/invocations \
  -H "Content-Type: application/json" \
  -d '{"inputs": [[[ [/* your 224x224 */] ]]] }'
```

* The payload shape must match your signature (here `NCHW` as `float32`).

**Docker (optional)**

```bash
mlflow models build-docker -m "models:/brain_mri_classifier/Production" -n mri:latest
docker run -p 8080:8080 mri:latest
```

---



### 🔹 How versions are created

* A **registered model** is like a “folder” (e.g., `brain_mri_classifier`).
* Every time you **register/log a new model** to that name:

  ```python
  mlflow.pytorch.log_model(..., registered_model_name="brain_mri_classifier")
  ```

  MLflow **automatically increments the version**:

  * First one → `Version 1`
  * Next one → `Version 2`
  * etc.
* **You cannot manually set the version number**. The registry handles that.

---


### 🔹 Stages

Each version starts in stage **"None"**.
You can promote them:

* `"Staging"` → testing environment
* `"Production"` → live
* `"Archived"` → old models you don’t want used anymore

## 1) Stages are deprecated → use **aliases** (or explicit versions)

* The new Model Registry UI hides Stages (“Production”, “Staging”). MLflow recommends **model version aliases** instead (e.g., `champion`, `production`, `staging`). You then reference models as:

  * By **alias**: `models:/brain_mri_classifier@production`
  * By **version**: `models:/brain_mri_classifier/2`
  * By **latest**: `models:/brain_mri_classifier/latest` (per flavor support)
    Docs & migration notes: ([mlflow.org][1])

### Set an alias (once per version)

```python
from mlflow import MlflowClient
client = MlflowClient()

# point alias "production" to version 2
client.set_registered_model_alias("brain_mri_classifier", "production", "2")

# (later) roll forward: repoint alias to version 3
client.set_registered_model_alias("brain_mri_classifier", "production", "3")
```


### Serve using the alias (or a version)

```bash
# make sure the client knows your tracking/registry server
export MLFLOW_TRACKING_URI=http://127.0.0.1:5000

# serve via alias (recommended)
mlflow models serve -m "models:/brain_mri_classifier@production" -h 0.0.0.0 -p 8080

# or serve a specific version
mlflow models serve -m "models:/brain_mri_classifier/2" -h 0.0.0.0 -p 8080
```



most teams **don’t** ship `mlflow models serve` to customers. They use it for **local testing / quick internal demos**, then deploy a **FastAPI (or Flask) microservice** that *loads the model from the MLflow Registry* (by **alias**), containerize it, and run it behind an API gateway with logging, auth, and autoscaling.

Here’s the landscape + a practical template you can drop in:

# What’s common in MLOps

**Good for local & CI smoke tests**

* `mlflow models serve` (or `build-docker`)
  Pros: 1-command, schema validation, works with any MLflow flavor.
  Cons: limited control over auth, routing, logging/metrics, batching, GPU pinning, multi-model, and custom business logic. Not ideal for production APIs.

**Most common for production**

* **FastAPI** (+ Uvicorn/Gunicorn) service that:

  1. Resolves your model from **MLflow Registry by alias** (e.g., `@production`).
  2. Does custom **pre/post-processing** (images, normalization, thresholds).
  3. Emits **structured logs/metrics**; supports **auth**, **tracing**, **health checks**.
  4. Runs in Docker/Kubernetes with autoscaling.

**Platformy options (when you need fleet scale)**

* KServe/Seldon, BentoML, TorchServe, vLLM/TF-Serving, Triton.
  MLflow remains your **registry of truth**; serving is handled by the platform.

---

# A production-ready FastAPI template (loads from MLflow Registry alias)

> Works with your earlier **pyfunc** wrapper or a pure PyTorch flavor. Uses aliasing (stages deprecated). GPU optional.

```python
# app.py
import os
import base64
import io
import pandas as pd
from fastapi import FastAPI
from pydantic import BaseModel
from PIL import Image
import mlflow

# 1) Point to your tracking/registry
MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI", "http://127.0.0.1:5000")
MODEL_URI = os.getenv("MODEL_URI", "models:/brain_mri_classifier@production")  # alias!

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# 2) Load once at startup (pyfunc recommended)
app = FastAPI(title="MRI Classifier Service")

@app.on_event("startup")
def load_model():
    global model
    model = mlflow.pyfunc.load_model(MODEL_URI)  # loads your PythonModel wrapper
    # If you logged a torch flavor instead:
    # import mlflow.pytorch as mpt
    # model = mpt.load_model(MODEL_URI).eval()

class PredictB64(BaseModel):
    image_b64: str

def pil_from_b64(b64: str) -> Image.Image:
    return Image.open(io.BytesIO(base64.b64decode(b64)))

@app.get("/health")
def health():
    return {"status": "ok", "model_uri": MODEL_URI}

@app.post("/predict")
def predict(req: PredictB64):
    # pyfunc expects a DataFrame by design
    df = pd.DataFrame({"image_b64": [req.image_b64]})
    preds = model.predict(df)  # np.ndarray or DataFrame (your wrapper decides)
    # convert to python types for JSON
    return {"predictions": preds.tolist()}
```

**Run locally**

```bash
export MLFLOW_TRACKING_URI=http://127.0.0.1:5000
export MODEL_URI="models:/brain_mri_classifier@production"
uvicorn app:app --host 0.0.0.0 --port 8080
```

**Dockerfile**

```dockerfile
FROM python:3.11-slim
WORKDIR /app
# Pin the same deps you logged with the model
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt
COPY app.py .
ENV MLFLOW_TRACKING_URI=http://mlflow:5000
ENV MODEL_URI=models:/brain_mri_classifier@production
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8080"]
```

**requirements.txt (align with what you logged)**

```
mlflow==2.14.1
torch==2.3.1
torchvision
pillow
pandas
fastapi
uvicorn
```

> **Rollouts:** when you train a better model, register it → **set alias**:
>
> ```python
> from mlflow import MlflowClient
> MlflowClient().set_registered_model_alias("brain_mri_classifier", "production", "3")
> ```
>
> Your service can either:
>
> * **Re-deploy** (cleanest), or
> * Expose a `/reload` admin endpoint that calls `mlflow.pyfunc.load_model(MODEL_URI)` again, or
> * Periodically re-resolve the alias on a timer.

---

# Why this beats `mlflow models serve` in prod

* **Pre/Post-processing**: MRI normalization, resizing, ensembles, thresholds.
* **Infra hooks**: request IDs, auth (JWT/API keys), rate limits, CORS.
* **Observability**: Prometheus/OTel metrics & traces, structured logs.
* **GPU control**: explicit `.to("cuda")` paths if you load a torch flavor.
* **Multi-endpoint**: `/predict`, `/metadata`, `/explain`, `/health`.
* **Safe rollouts**: alias-switch + blue/green or canary at the API gateway.

---

# When to still use `mlflow models serve`

* Quick **manual QA** of a just-logged model.
* **CI smoke tests** to ensure the model artifact is runnable.
* Prototyping the input schema (`signature` / `input_example`).

---

# Minimal checklist for your MRI classifier

1. **Registry**

   * Log model with `registered_model_name=...`, **signature**, **input\_example**, **pinned requirements**.
   * Use **aliases** (`production`, `staging`).

2. **Service**

   * FastAPI + pyfunc wrapper (you already have it).
   * Health check, error handling, request validation.

3. **Container & Run**

   * Docker, Uvicorn/Gunicorn, env vars for `MLFLOW_TRACKING_URI` and `MODEL_URI`.
   * GPU node? Add `nvidia/cuda` base + `--gpus` in Docker run / k8s device plugin.

4. **Ops**

   * Logs/metrics/traces, autoscaling (HPA), gateway (auth, TLS), alerts.
   * Rollout via **alias switch**; rollback = point alias back.



## What `mlflow models serve` actually does

When you run:

```bash
mlflow models serve -m "<MODEL_URI>" -p 8080
```

MLflow:

1. **Resolves your model URI** (run path, registry version/alias, local folder).
2. **Loads the model’s “flavor”** (most commonly `pyfunc`, but could be `pytorch`, `sklearn`, etc.; all are exposed via the `pyfunc` interface). ([mlflow.org][1])
3. **Restores the model’s environment** if requested (based on the saved `conda.yaml` / pip requirements). For non-local env managers, MLflow will **spin up a separate “Model Scoring Server” process** in that environment and proxy requests to it. ([mlflow.org][2])
4. **Starts a local HTTP scoring server** implementing MLflow’s standard REST API (not your app server):

   * `GET /ping` for health
   * `POST /invocations` for predictions (accepts JSON or CSV in the MLflow scoring schema). ([mlserver.readthedocs.io][3], [mlflow.org][1])

This CLI is ideal for **local testing / smoke tests** and “does what it says on the tin”: runs a small scoring server bound to your model and its saved dependencies. ([mlflow.org][4])

### Notes

* Request/response format follows the **pyfunc protocol** (pandas-style JSON like `dataframe_split`, or CSV). Your `pyfunc` wrapper’s `predict()` is what gets called. ([mlflow.org][5])
* If you logged an **input example**, MLflow even generates a **serving payload example** (`serving_input_example.json`). ([mlflow.org][1])

---

## Is there a “build-docker” option?

Yes. MLflow ships a **Docker image builder** for models:

```bash
mlflow models build-docker \
  -m "<MODEL_URI>" \
  -n mymodel:latest \
  --enable-mlserver   # optional: use Seldon’s MLServer runtime
```

* The image’s entrypoint serves your model on port **8080** using the **python\_function (pyfunc) scoring server** (or **MLServer** if you pass `--enable-mlserver`). ([mlflow.org][6], [mlserver.readthedocs.io][3])
* You can then run: `docker run -p 8080:8080 mymodel:latest` and hit `/invocations`. ([mlflow.org][6])

MLflow also documents broader deployment targets (local server, Kubernetes with the built image, cloud endpoints, etc.). ([mlflow.org][7])

---

## When to use which

* **`mlflow models serve`**: quick local runs, CI smoke tests, ad-hoc validation. ([mlflow.org][4])
* **`mlflow models build-docker`**: package the exact scoring server + deps as a portable image (great for K8s). ([mlflow.org][7])
* **Custom FastAPI/Flask service**: production APIs needing auth, custom pre/post-processing, observability, batching, GPU control (most teams do this; the service *loads from the MLflow Registry by alias*). MLflow remains your **model packaging + registry**, not necessarily your prod web stack.

If you want, I can give you a tiny `Makefile` + `Dockerfile` pairing both workflows (plain `serve` vs image via `build-docker`) wired to your current registry alias.

[1]: https://mlflow.org/docs/latest/ml/model/?utm_source=chatgpt.com "MLflow Models"
[2]: https://www.mlflow.org/docs/2.17.2/_modules/mlflow/pyfunc.html?utm_source=chatgpt.com "mlflow.pyfunc"
[3]: https://mlserver.readthedocs.io/en/latest/examples/mlflow/README.html?utm_source=chatgpt.com "Serving MLflow models — MLServer Documentation"
[4]: https://mlflow.org/docs/latest/ml/deployment/deploy-model-locally/?utm_source=chatgpt.com "Deploy MLflow Model as a Local Inference Server"
[5]: https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html?utm_source=chatgpt.com "mlflow.pyfunc"
[6]: https://mlflow.org/docs/latest/cli.html?utm_source=chatgpt.com "Command-Line Interface"
[7]: https://mlflow.org/docs/latest/ml/deployment/deploy-model-to-kubernetes/?utm_source=chatgpt.com "Deploy MLflow Model to Kubernetes"


Great question. Two parts:

# 1) Auto-capturing `pip_requirements` when you call `mlflow.pytorch.log_model`

* **Do nothing**: if you *don’t* pass `pip_requirements` / `conda_env` / `python_env`, MLflow will **infer** the minimal requirements for the model flavor and write them into the model’s `requirements.txt`, `python_env.yaml`, and `conda.yaml`. Adding an `input_example` improves this inference because MLflow performs a dry-run predict to see imports in action. ([mlflow.org][1])

* **Add to what MLflow inferred**: pass `extra_pip_requirements=[...]` to append packages (e.g., your preprocessing libs) without overriding the inferred set. ([mlflow.org][1])

* **Fully control deps**: pass `pip_requirements=[...]` (explicit list or path to a `requirements.txt`) **or** `conda_env=...` / `python_env=...` to take full control. (Don’t pass these together; they’re mutually exclusive.) Many flavors—including PyTorch and PyFunc—support this pattern. ([mlflow.org][2])

* **From “current project” explicitly**: you can precompute with:

  ```python
  from mlflow.models import infer_pip_requirements
  reqs = infer_pip_requirements(model_uri="/path/to/temp_model_dir", flavor="pytorch")
  mlflow.pytorch.log_model(model, artifact_path="model", pip_requirements=reqs)
  ```

  (If inference hits something unusual, MLflow falls back to flavor defaults; you’ll see a warning.) ([mlflow.org][1], [community.databricks.com][3])

* **Lock all transitive deps** (for reproducibility): set

  ```bash
  export MLFLOW_LOCK_MODEL_DEPENDENCIES=true
  ```

  MLflow will resolve and pin the full tree (uses `uv` if present) when logging the model. ([mlflow.org][1])

Quick example you can reuse:

```python
mlflow.pytorch.log_model(
    pytorch_model=model,
    artifact_path="model",
    # leave pip_requirements unset to let MLflow infer,
    extra_pip_requirements=["pillow==10.4.0"],  # add your extras
    input_example=x_example,                     # helps inference accuracy
    signature=signature,
)
```

# 2) “Docker option” for models

There isn’t a `docker=` switch in `log_model`. Docker packaging is a **separate step**:

* **CLI**:

  ```bash
  mlflow models build-docker -m "models:/brain_mri_classifier@production" -n mri:latest
  docker run -p 8080:8080 mri:latest
  ```

  Builds an image whose entrypoint serves your model on port 8080 (pyfunc server by default; add `--enable-mlserver` to use MLServer). ([mlflow.org][4])

* **Python API**:

  ```python
  import mlflow
  mlflow.models.build_docker(
      model_uri="models:/brain_mri_classifier@production",
      name="mri:latest",
      enable_mlserver=False,  # True to use MLServer
  )
  ```

  (Handy for CI/CD pipelines and k8s deploys.) ([mlflow.org][5])

# TL;DR

* **Easiest**: let MLflow infer deps; add `input_example`; optionally append `extra_pip_requirements`. ([mlflow.org][1])
* **Strict reproducibility**: flip `MLFLOW_LOCK_MODEL_DEPENDENCIES=true`. ([mlflow.org][1])
* **Docker** is done **after** logging via `mlflow models build-docker` (CLI or Python). ([mlflow.org][4])

If you want, I can generate a tiny script that logs your current model, prints the inferred requirements, and builds the Docker image in one go.

[1]: https://mlflow.org/docs/latest/ml/model/dependencies/ "Managing Dependencies in MLflow Models | MLflow"
[2]: https://mlflow.org/docs/2.3.0/python_api/mlflow.pytorch.html?utm_source=chatgpt.com "mlflow.pytorch — MLflow 2.3.0 documentation"
[3]: https://community.databricks.com/t5/machine-learning/how-to-fix-quot-warning-mlflow-utils-environment-quot-when-run/td-p/15930?utm_source=chatgpt.com "How to fix \"WARNING mlflow.utils.environment\" when..."
[4]: https://mlflow.org/docs/latest/cli.html?utm_source=chatgpt.com "Command-Line Interface"
[5]: https://mlflow.org/docs/latest/ml/deployment/deploy-model-to-kubernetes/?utm_source=chatgpt.com "Deploy MLflow Model to Kubernetes"


That’s a very sharp question 🙂 — it gets right at how **MLflow controls Docker images**.

---

## 🔹 How `mlflow models build-docker` works

* You **don’t pass your own Dockerfile** directly.
* MLflow auto-generates a **standard Dockerfile** for you, based on:

  * The **model flavor** (e.g. `pyfunc`, `pytorch`, `sklearn`).
  * The **environment spec** that was saved when you logged the model (`conda.yaml`, `python_env.yaml`, or `requirements.txt`).
* That Dockerfile installs:

  1. Base image (`python:3.11-slim` or similar).
  2. Your pinned dependencies from the model’s environment files.
  3. MLflow itself + the scoring server.
  4. Entrypoint: runs `mlflow models serve` (or `mlserver` if you pass `--enable-mlserver`).

So when you run:

```bash
mlflow models build-docker \
    -m models:/brain_mri_classifier@production \
    -n brain_mri:latest
```

MLflow generates:

* `Dockerfile` (with the base + env).
* `requirements.txt` / `conda.yaml` from the model artifact.
* Then runs `docker build` behind the scenes.

---

## 🔹 Can you customize it?

Yes, but **not inline**. Options:

1. **Edit the logged model’s environment**

   * When you `log_model`, supply your own `pip_requirements` or `conda_env`.
   * Those files are baked into the auto-Dockerfile.
   * This is the standard way to say “what goes into the Docker image.”

2. **Extend the generated image**

   * Let MLflow build the image (`mlflow models build-docker -n base_model:latest`).
   * Then write your own `Dockerfile` that uses it as the base:

     ```dockerfile
     FROM base_model:latest
     RUN apt-get update && apt-get install -y ffmpeg
     COPY extras/ /app/extras/
     ```
   * Build your extended image with `docker build`.
   * This is the common pattern if you need system packages (e.g., OpenCV libs, CUDA, ffmpeg).

3. **Programmatic control**

   * MLflow exposes a `build_docker()` Python API (same as CLI).
   * You still don’t give it a custom Dockerfile, but you can control some knobs (image name, whether to use MLServer).

---

## 🔹 Best practice in MLOps

* **Reproducibility**: lock your environment (`MLFLOW_LOCK_MODEL_DEPENDENCIES=true`) at log time → guarantees the same deps in the Docker image.
* **Custom runtime needs** (system libs, configs, GPU base images): extend MLflow’s generated image with your own Dockerfile (pattern #2).
* **Production serving**: many teams only use `mlflow models build-docker` to generate a **base scoring image**, then wrap it in a FastAPI/Flask microservice that does auth/logging, and extend the Dockerfile accordingly.

---

✅ **Summary:**

* You don’t supply a Dockerfile in `mlflow models build-docker`.
* It’s generated automatically from the model’s saved env (`conda.yaml`, `requirements.txt`, etc.).
* To customize: either (a) set `pip_requirements`/`conda_env` at log time, or (b) extend the generated image with your own Dockerfile.

---

Would you like me to show you **what the generated Dockerfile looks like** for a PyTorch model, so you know exactly what MLflow puts in there before you extend it?


Perfect question 👍 — this is one of the least obvious parts of MLflow.

---

## 🔹 What happens when you run

```bash
mlflow models serve -m <MODEL_URI> -p 8080
```

### Step 1. Resolve the model

* MLflow resolves `<MODEL_URI>` to a local path (download from registry or use run artifact).

### Step 2. Inspect environment files

* Every logged MLflow model has an **environment spec** saved under its model directory (`MLmodel` file references it):

  * `conda.yaml`
  * or `python_env.yaml`
  * or `requirements.txt`

This was created when you logged the model (either inferred or user-provided).
That spec tells MLflow **which packages** to install for serving.

### Step 3. Launch a scoring process

* By default MLflow **does not change your current shell environment**.
* Instead, it starts a **new subprocess** called the **MLflow Scoring Server** in the right environment.
* The environment is prepared by:

  * **pip** if you used `python_env.yaml` or `requirements.txt` (virtualenv backend).
  * **conda** if you used `conda.yaml` (conda backend).

Which backend gets used depends on what’s available and how the model was logged:

* MLflow checks environment files in this order: `python_env.yaml` → `conda.yaml` → `requirements.txt`.
* If it finds `conda.yaml`, it tries to use conda.
* If `python_env.yaml`, it uses MLflow’s internal **virtualenv backend** (pip-based).
* If none, it falls back to your current Python env.

So **yes**: it may create a conda env, or a virtualenv, and then `pip install -r` inside it.

### Step 4. Run the scoring server

* Inside that isolated env, MLflow starts a small REST server:

  * `GET /ping` → health check
  * `POST /invocations` → runs your model’s `pyfunc.predict()` on the payload
* It communicates with your outer process through HTTP (on port 8080).

---

## 🔹 What’s installed in that env

* Exactly the packages listed in the env files (`conda.yaml` / `requirements.txt` / `python_env.yaml`).
* Always includes `mlflow` itself and whatever flavor library your model needs (e.g. `torch`, `sklearn`).
* If you had `pip_requirements` pinned when logging → those are pip installed.
* If you set `MLFLOW_LOCK_MODEL_DEPENDENCIES=true` → it resolves and locks versions, then uses those in serving.

---

## 🔹 Where it runs

* By default, the envs are created under MLflow’s cache directory (`~/.mlflow/envs/...`).
* Each unique set of requirements creates its own cached environment.
* Next time you `mlflow models serve` the same model, MLflow reuses that env.

---

## 🔹 TL;DR

* **Yes, MLflow installs requirements automatically** — pip or conda depending on what env file was saved.
* It runs the model inside an **isolated scoring process** with that env, not in your current Python session.
* That’s why `mlflow models serve` “just works” even if your shell env doesn’t have the right deps.

---

👉 Want me to show you a real example of the **generated `conda.yaml`** and the **subprocess command** MLflow runs when serving a PyTorch model? That way you can see exactly how your `.pth` gets wrapped.


In [6]:
import mlflow
from mlflow.models import infer_signature
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
import numpy as np

In [7]:
# Generate synthetic dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=3, 
                         n_informative=5, n_redundant=10, 
                         n_clusters_per_class=2, random_state=1, 
                         weights=[0.5, 0.40, 0.1])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define model parameters
lr_params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "random_state": 8888,
}

gb_params = {
    "n_estimators": 100,
    "learning_rate": 0.1,
    "random_state": 8888,
    "max_depth": 3
}

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [9]:
def train_and_evaluate_model(model, model_name, params, X_train, X_test, y_train, y_test):
    with mlflow.start_run(run_name=f"{model_name}"):
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        report_dict = classification_report(y_test, y_pred, output_dict=True)
        
        # Log parameters
        mlflow.log_params(params)
        
        # Log overall metrics
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("weighted_precision", report_dict['weighted avg']['precision'])
        mlflow.log_metric("weighted_recall", report_dict['weighted avg']['recall'])
        mlflow.log_metric("weighted_f1", report_dict['weighted avg']['f1-score'])
        mlflow.log_metric("macro_avg_precision", report_dict['macro avg']['precision'])
        mlflow.log_metric("macro_avg_recall", report_dict['macro avg']['recall'])
        mlflow.log_metric("macro_avg_f1", report_dict['macro avg']['f1-score'])
        
        # Log per-class metrics
        for class_idx in range(len(report_dict) - 3):  # -3 to exclude the averages
            class_name = str(class_idx)
            if class_name in report_dict:
                mlflow.log_metric(f"class_{class_name}_precision", report_dict[class_name]['precision'])
                mlflow.log_metric(f"class_{class_name}_recall", report_dict[class_name]['recall'])
                mlflow.log_metric(f"class_{class_name}_f1", report_dict[class_name]['f1-score'])
                mlflow.log_metric(f"class_{class_name}_support", report_dict[class_name]['support'])
        
        # Log the classification report as a text artifact
        # with open("classification_report.txt", "w") as f:
        #     f.write(classification_report(y_test, y_pred))
        mlflow.log_artifact("classification_report.txt")
        
        # Log the model
        signature = infer_signature(X_train, model.predict(X_train))
        model_info = mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path=f"{model_name.lower()}_model",
            signature=signature,
            input_example=X_train,
            registered_model_name=f"{model_name.lower()}-model",
        )
        
        print(f"\n{model_name} Results:")
        print(classification_report(y_test, y_pred))
        
        return model_info

In [10]:
# Set MLflow tracking URI
host = "127.0.0.1"
port = 8080

mlflow.set_tracking_uri(uri=f"http://{host}:{port}")
print(f"MLflow tracking URI set to: {mlflow.get_tracking_uri()}")

# Create a new MLflow Experiment with a unique name
experiment_name = "Model Comparison New"
try:
    # Try to get the experiment by name
    experiment = mlflow.get_experiment_by_name(experiment_name)
    
    # If experiment doesn't exist, create it
    if experiment is None:
        experiment_id = mlflow.create_experiment(experiment_name)
        print(f"Created new experiment: {experiment_name} with ID: {experiment_id}")
    else:
        print(f"Using existing experiment: {experiment_name}")
    
    # Set the experiment as active
    mlflow.set_experiment(experiment_name)
except Exception as e:
    print(f"Error setting up experiment: {e}")
    # Fallback to using the default experiment
    mlflow.set_experiment("Default")

# Train and evaluate LogisticRegression
lr = LogisticRegression(**lr_params)
lr_model_info = train_and_evaluate_model(lr, "LogisticRegression", lr_params, 
                                      X_train, X_test, y_train, y_test)

# Train and evaluate GradientBoostingClassifier
gb = GradientBoostingClassifier(**gb_params)
gb_model_info = train_and_evaluate_model(gb, "GradientBoosting", gb_params, 
                                      X_train, X_test, y_train, y_test)

MLflow tracking URI set to: http://127.0.0.1:8080
Error setting up experiment: API request to http://127.0.0.1:8080/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=Model+Comparison+New (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x763d5c5baff0>: Failed to establish a new connection: [Errno 111] Connection refused'))


MlflowException: API request to http://127.0.0.1:8080/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=Default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x763d5c8933e0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
# Compare predictions
lr_predictions = lr.predict(X_test)
gb_predictions = gb.predict(X_test)

# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'actual_class': y_test,
    'lr_predicted': lr_predictions,
    'gb_predicted': gb_predictions
})

print("\nPrediction Comparison (first 5 samples):")
print(comparison_df.head())